In [4]:
import (
	"fmt"

	"gonum.org/v1/hdf5"
)

func main() {
	fmt.Println("=== go-hdf5 ===")
	version, err := hdf5.LibVersion()
	if err != nil {
		fmt.Printf("** error ** %s\n", err)
		return
	}
	fmt.Printf("=== version: %s", version)
	fmt.Println("=== bye.")
}

=== go-hdf5 ===
=== version: 1.10.4=== bye.


In [5]:
import (
	"fmt"

	"gonum.org/v1/hdf5"
)

const (
	fname  string = "SDScompound.h5"
	dsname string = "ArrayOfStructures"
	mbr1   string = "A_name"
	mbr2   string = "B_name"
	mbr3   string = "C_name"
	length uint   = 10
	rank   int    = 1
)

type s1Type struct {
	a int
	b float32
	c float64
	d [3]int
	e string
}

type s2Type struct {
	c float64
	a int
}

func main() {

	// initialize data
	// s1 := make([]s1_t, LENGTH)
	// for i:=0; i<LENGTH; i++ {
	// 	s1[i] = s1_t{a:i, b:float32(i*i), c:1./(float64(i)+1)}
	// }
	// fmt.Printf(":: data: %v\n", s1)
	s1 := [length]s1Type{}
	for i := 0; i < int(length); i++ {
		s1[i] = s1Type{
			a: i,
			b: float32(i * i),
			c: 1. / (float64(i) + 1),
			d: [...]int{i, i * 2, i * 3},
			e: fmt.Sprintf("--%d--", i),
		}
		//s1[i].d = []float64{float64(i), float64(2*i), 3.*i}}
	}
	fmt.Printf(":: data: %v\n", s1)

	// create data space
	dims := []uint{length}
	space, err := hdf5.CreateSimpleDataspace(dims, nil)
	if err != nil {
		panic(err)
	}

	// create the file
	f, err := hdf5.CreateFile(fname, hdf5.F_ACC_TRUNC)
	if err != nil {
		panic(err)
	}
	defer f.Close()
	fmt.Printf(":: file [%s] created (id=%d)\n", fname, f.ID())

	// create the memory data type
	dtype, err := hdf5.NewDatatypeFromValue(s1[0])
	if err != nil {
		panic("could not create a dtype")
	}

	// create the dataset
	dset, err := f.CreateDataset(dsname, dtype, space)
	if err != nil {
		panic(err)
	}
	fmt.Printf(":: dset (id=%d)\n", dset.ID())

	// write data to the dataset
	fmt.Printf(":: dset.Write...\n")
	err = dset.Write(&s1)
	if err != nil {
		panic(err)
	}
	fmt.Printf(":: dset.Write... [ok]\n")

	// release resources
	dset.Close()
	f.Close()

	// open the file and the dataset
	f, err = hdf5.OpenFile(fname, hdf5.F_ACC_RDONLY)
	if err != nil {
		panic(err)
	}
	dset, err = f.OpenDataset(dsname)
	if err != nil {
		panic(err)
	}

	// read it back into a new slice
	s2 := make([]s1Type, length)
	err = dset.Read(&s2)
	if err != nil {
		panic(err)
	}

	// display the fields
	fmt.Printf(":: data: %v\n", s2)

	// release resources
	dset.Close()
	f.Close()
}

:: data: [{0 0 1 [0 0 0] --0--} {1 1 0.5 [1 2 3] --1--} {2 4 0.3333333333333333 [2 4 6] --2--} {3 9 0.25 [3 6 9] --3--} {4 16 0.2 [4 8 12] --4--} {5 25 0.16666666666666666 [5 10 15] --5--} {6 36 0.14285714285714285 [6 12 18] --6--} {7 49 0.125 [7 14 21] --7--} {8 64 0.1111111111111111 [8 16 24] --8--} {9 81 0.1 [9 18 27] --9--}]
:: file [SDScompound.h5] created (id=72057594037927936)
:: dset (id=360287970189639680)
:: dset.Write...


panic: runtime error: cgo argument has Go pointer to Go pointer

goroutine 1 [running]:
gonum.org/v1/hdf5.(*Dataset).WriteSubset.func1(0x4a4b80?, 0xc00011a000?, 0x4b7ea0?, 0xc00007e340?, 0xc000061e30?)
	/home/jmbenlloch/go/pkg/mod/gonum.org/v1/hdf5@v0.0.0-20210714002203-8c5d23bc6946/h5d_dataset.go:147 +0x4a
gonum.org/v1/hdf5.(*Dataset).WriteSubset(0x4dbab8?, {0x4a4b80?, 0xc00011a000}, 0x0, 0x0)
	/home/jmbenlloch/go/pkg/mod/gonum.org/v1/hdf5@v0.0.0-20210714002203-8c5d23bc6946/h5d_dataset.go:147 +0x245
gonum.org/v1/hdf5.(*Dataset).Write(...)
	/home/jmbenlloch/go/pkg/mod/gonum.org/v1/hdf5@v0.0.0-20210714002203-8c5d23bc6946/h5d_dataset.go:154
main.main()
	 [[ Cell [5] Line 81 ]] /tmp/gonb_484cc0f1/main.go:82 +0x465
exit status 2
